<a href="https://colab.research.google.com/github/pawarad/ADS/blob/master/Fast_parapred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Biopython

     |████████████████████████████████| 2.3MB 2.8MB/s 


In [2]:
from Bio.PDB import *
from Bio.PDB.Model import Model
from Bio.PDB.Structure import Structure
import numpy as np
import Bio.PDB
import pickle
import pandas as pd

In [3]:
def residue_in_contact_with(res, ab_search, dist):
    return any(len(ab_search.search(a.coord, dist)) > 0
               for a in res.get_unpacked_list())

In [4]:
def aaMap(_aa):
    aa_df = pd.DataFrame([['Alanine','Ala','A'],
                        ['Arginine','Arg','R'],
                        ['Asparagine','Asn','N'],
                        ['Aspartate','Asp','D'],
                        ['Cysteine','Cys','C'],
                        ['Glutamate','Glu','E'],
                        ['Glutamine','Gln','Q'],
                        ['Glycine','Gly','G'],
                        ['Histidine','His','H'],
                        ['Histidine_D','HID','H'],
                        ['Histidine_E','HIE','H'],
                        ['Histidine_P','HIP','H'],
                        ['Isoleucine','Ile','I'],
                        ['Leucine','Leu','L'],
                        ['Lysine','Lys','K'],
                        ['Methionine','Met','M'],
                        ['Phenylalanine','Phe','F'],
                        ['Proline','Pro','P'],
                        ['Serine','Ser','S'],
                        ['Threonine','Thr','T'],
                        ['Tryptophan','Trp','W'],
                        ['Tyrosine','Tyr','Y'],
                        ['Valine','Val','V']],columns=['Full','3','1'])
    aa_df['3'] = aa_df['3'].str.upper()
    _aa = _aa.upper()
    if len(_aa)==1:
        toret = aa_df.loc[aa_df['1']==_aa,'3'].values
        if len(toret)>0:
            toret = toret[0]
        else:
            return None
    else:
        toret = aa_df.loc[aa_df['3']==_aa,'1'].values
        if len(toret)>0:
            toret = toret[0]
        else:
            return None
    return toret

def maxi_len(all_variables):
    max_length = 0
    for i in range(len(all_variables)):
        length = len(all_variables[i])
        max_length = max(max_length,length)
    #     max_length = length
    return max_length


def ext_epitope(H_chain,L_chain,A_chain,pdb):
    p = PDBParser()
    structure = p.get_structure('X', "/content/drive/My Drive/Peritia_Fast-Parapred/cleaned_pdb_shweta/"+pdb+'.pdb')
    print(pdb)
    ## construct a Model with residues of Heavy and Light chain of the pdb
    model = structure[0]
    model1 = Model(0)
    model1.add(model[H_chain]) ## heavy chain
    model1.add(model[L_chain]) ## Light chain
    
#     print(list(model1))

    ## construct another Model with residues of Antigen chain of the pdb
    model2 = Model(1)
    for i in A_chain:
        model2.add(model[i])
#     print(list(model2))
    
    ## Search for nearest atoms within the Antibody(Heavy/Light chain)
    ab_search = NeighborSearch(Selection.unfold_entities(model1, 'A'))

    epitope = []
    for r in model2:
        for j in r:
            if residue_in_contact_with(j, ab_search,5) == True:
                epitope.append(j)

    epi_residues =[]
    epi_res_number = []
    for i in epitope:
        tags = i.id
    #     print(tags)
        if tags[0] == " ":
            res_name = i.get_resname() 
            res_abbre = aaMap(res_name)
            epi_residues.append(res_abbre)
            epi_res_number.append(str(tags[1])+str(tags[2]))
#     print("epi_res",epi_residues) 
#     print("epi_res_number",epi_res_number )

    # print("tags",sorted(epitope))

    epi_search = NeighborSearch(Selection.unfold_entities(epitope, 'A'))

    ext_epi =[]
    for r in model2:
        for j in r:
            if residue_in_contact_with(j, epi_search,10) == True:
                ext_epi.append(j)

    ext_epi_residues = []
    ext_epi_res_number = []
    for i in ext_epi:
        tags = i.id
    #     print("tags",sorted(tags[1]))
        if tags[0] == " ":
            res_name = i.get_resname() 
            res_abbre = aaMap(res_name)
            ext_epi_residues.append(res_abbre)
    #         print(i,res_name)
            ext_epi_res_number.append(str(tags[1])+str(tags[2]))
#     print("ext_epi_res_number",ext_epi_res_number)
#     print("ext_epi_res",ext_epi_residues)


    return epi_residues, epi_res_number, ext_epi_res_number, ext_epi_residues



def ext_epitope_sorted(H_chain,L_chain,A_chain,pdb):
    p = PDBParser()
    structure = p.get_structure('X', "/content/drive/My Drive/Peritia_Fast-Parapred/cleaned_pdb_shweta/"+pdb+'.pdb')
    print(pdb)
    ## construct a Model with residues of Heavy and Light chain of the pdb
    model = structure[0]
    model1 = Model(0)
    model1.add(model[H_chain]) ## heavy chain
    model1.add(model[L_chain]) ## Light chain
    
#     print(list(model1))

    ## construct another Model with residues of Antigen chain of the pdb
    model2 = Model(1)
    for i in A_chain:
        model2.add(model[i])
#     print(list(model2))
    
    ## Search for nearest atoms within the Antibody(Heavy/Light chain)
    ab_search = NeighborSearch(Selection.unfold_entities(model1, 'A'))

    epitope = []
    for r in model2:
        for j in r:
            if residue_in_contact_with(j, ab_search,5) == True:
                epitope.append(j)

    epi_residues =[]
    epi_res_number = []
    for i in sorted(epitope):
        tags = i.id
    #     print(tags)
        if tags[0] == " ":
            res_name = i.get_resname() 
            res_abbre = aaMap(res_name)
            epi_residues.append(res_abbre)
            epi_res_number.append(str(tags[1])+str(tags[2]))
#     print("epi_res",epi_residues) 
#     print("epi_res_number",epi_res_number )

    # print("tags",sorted(epitope))

    epi_search = NeighborSearch(Selection.unfold_entities(epitope, 'A'))

    ext_epi =[]
    for r in model2:
        for j in r:
            if residue_in_contact_with(j, epi_search,10) == True:
                ext_epi.append(j)

    ext_epi_residues = []
    ext_epi_res_number = []
    for i in sorted(ext_epi):
        tags = i.id
    #     print("tags",sorted(tags[1]))
        if tags[0] == " ":
            res_name = i.get_resname() 
            res_abbre = aaMap(res_name)
            ext_epi_residues.append(res_abbre)
    #         print(i,res_name)
            ext_epi_res_number.append(str(tags[1])+str(tags[2]))
#     print("ext_epi_res_number",ext_epi_res_number)
#     print("ext_epi_res",ext_epi_residues)


    return epi_residues, epi_res_number, ext_epi_res_number, ext_epi_residues


aa_s = "CSTPAGNDEQHRKMILVFYWX"
def one_to_number(res_str):
    return [aa_s.index(r) for r in res_str]

def aa_features():
    # Meiler's features
    prop1 = [[1.77, 0.13, 2.43,  1.54,  6.35, 0.17, 0.41],
             [1.31, 0.06, 1.60, -0.04,  5.70, 0.20, 0.28],
             [3.03, 0.11, 2.60,  0.26,  5.60, 0.21, 0.36],
             [2.67, 0.00, 2.72,  0.72,  6.80, 0.13, 0.34],
             [1.28, 0.05, 1.00,  0.31,  6.11, 0.42, 0.23],
             [0.00, 0.00, 0.00,  0.00,  6.07, 0.13, 0.15],
             [1.60, 0.13, 2.95, -0.60,  6.52, 0.21, 0.22],
             [1.60, 0.11, 2.78, -0.77,  2.95, 0.25, 0.20],
             [1.56, 0.15, 3.78, -0.64,  3.09, 0.42, 0.21],
             [1.56, 0.18, 3.95, -0.22,  5.65, 0.36, 0.25],
             [2.99, 0.23, 4.66,  0.13,  7.69, 0.27, 0.30],
             [2.34, 0.29, 6.13, -1.01, 10.74, 0.36, 0.25],
             [1.89, 0.22, 4.77, -0.99,  9.99, 0.32, 0.27],
             [2.35, 0.22, 4.43,  1.23,  5.71, 0.38, 0.32],
             [4.19, 0.19, 4.00,  1.80,  6.04, 0.30, 0.45],
             [2.59, 0.19, 4.00,  1.70,  6.04, 0.39, 0.31],
             [3.67, 0.14, 3.00,  1.22,  6.02, 0.27, 0.49],
             [2.94, 0.29, 5.89,  1.79,  5.67, 0.30, 0.38],
             [2.94, 0.30, 6.47,  0.96,  5.66, 0.25, 0.41],
             [3.21, 0.41, 8.08,  2.25,  5.94, 0.32, 0.42],
             [0.00, 0.00, 0.00,  0.00,  0.00, 0.00, 0.00]]
    return np.array(prop1)

NUM_FEATURES = len(aa_s) + 7

def to_categorical(y, num_classes):
    """ Converts a class vector to binary class matrix. """
    new_y = torch.LongTensor(y)
    n = new_y.size()[0]
    categorical = torch.zeros(n, num_classes)
    arangedTensor = torch.arange(0, n)
    intaranged = arangedTensor.long()
    categorical[intaranged, new_y] = 1
    return categorical

def seq_to_one_hot(res_seq_one):
#     from keras.utils.np_utils import to_categorical
    ints = one_to_number(res_seq_one)
    new_ints = torch.LongTensor(ints)
    feats = torch.Tensor(aa_features()[new_ints])
    onehot = to_categorical(ints, num_classes=len(aa_s))
    return torch.cat((onehot, feats), axis=1)

def cdrseq_to_one_hot(res_seq_one,i):
#     from keras.utils.np_utils import to_categorical
    ints = one_to_number(res_seq_one)
    new_ints = torch.LongTensor(ints)
    feats = torch.Tensor(aa_features()[ints])
    onehot = to_categorical(ints, num_classes=len(aa_s))
    if i%6 == 0:
        ext_onehot = [1, 0, 0, 0, 0, 0]
    if i%6 == 1:
        ext_onehot = [0, 1, 0, 0, 0, 0]
    if i%6 == 2:
        ext_onehot = [0, 0, 1, 0, 0, 0]
    if i%6 == 3:
        ext_onehot = [0, 0, 0, 1, 0, 0]
    if i%6 == 4:
        ext_onehot = [0, 0, 0, 0, 1, 0]
    if i%6 == 5:
        ext_onehot = [0, 0, 0, 0, 0, 1]
    
    chain_encoding = torch.Tensor(ext_onehot)
    chain_encoding = chain_encoding.expand(onehot.shape[0], 6)
    concatenated = torch.cat((onehot, feats,chain_encoding), 1)

    return concatenated



In [5]:
from __future__ import print_function
from sklearn.model_selection import KFold

import numpy as np
# np.set_printoptions(threshold=np.nan)
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch import index_select

In [6]:
MAX_CDR_LENGTH = 38

MAX_EXT_AG_LENGTH = 288

use_cuda = torch.cuda.is_available()

epochs = 20

batch_size = 32

In [7]:
class AtrousSelf(nn.Module):
    def __init__(self):
        super(AtrousSelf, self).__init__()
        self.conv1 = nn.Conv1d(NUM_FEATURES, 64, 3, padding=1)  # a trous convolution
        self.conv2 = nn.Conv1d(64, 128, 3, padding=2, dilation=2)
        self.conv3 = nn.Conv1d(128, 256, 3, padding=4, dilation=4)
        self.bn1 = nn.BatchNorm1d(64)  # batch normalisation after first a trous convolution
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(256)
        self.bn4 = nn.BatchNorm1d(512)
        self.elu = nn.ReLU()
        self.dropout = nn.Dropout(0.15)
        self.dropout2 = nn.Dropout(0.5)
        self.fc = nn.Linear(512, 1, 1)
        self.softmax = nn.Softmax(dim=-1)
        self.lrelu = nn.LeakyReLU(0.2)

        self.aconv1 = nn.Conv1d(256, 1, 1)
        self.aconv2 = nn.Conv1d(256, 1, 1)

        for m in self.modules():
            self.weights_init(m)

    def weights_init(self, m):
        """
        :param m: Layer type
        :return: void. Performs parameter initialisation
        """
        if isinstance(m, nn.Conv1d):
            torch.nn.init.xavier_uniform(m.weight.data)
            m.bias.data.fill_(0.0)
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform(m.weight.data)
            m.bias.data.fill_(0.0)

    def forward(self, input, unpacked_masks):
        """
        :param input: antibody amino acid sequences
        :param unpacked_masks:
        :return: antibody binding probabilities
        """
        x=input

        unpacked_masks = torch.transpose(unpacked_masks, 1, 2)

        x = torch.transpose(x, 1, 2)
        x = self.conv1(x)

        #print("x after conv1", x.data.shape)
        x = torch.mul(x, unpacked_masks)
        x = self.bn1(x)
        x = self.elu(x)
        x = torch.mul(x, unpacked_masks)
        x = self.dropout(x)

        x = self.conv2(x)
        #print("x after conv2", x.data.shape)

        x = torch.mul(x, unpacked_masks)
        x = self.bn2(x)
        x = self.elu(x)
        x = torch.mul(x, unpacked_masks)

        x = self.dropout(x)

        x = self.conv3(x)
        #print("x after conv3", x.data.shape)

        x = torch.mul(x, unpacked_masks)
        x = self.bn3(x)
        x = self.elu(x)
        x = torch.mul(x, unpacked_masks)

        x = self.dropout(x)

        old = x

        #print("x", x.data.shape)

        w_1 = self.aconv1(x)
        #print("w_1", w_1.data.shape)

        w_2 = self.aconv2(x)
        #print("w_2", w_2.data.shape)


        w = self.lrelu(w_1 + torch.transpose(w_2, 1, 2))

        #print("w", w.data.shape)

        bias_mat = 1e9 * (unpacked_masks - 1.0)
        w = self.softmax(w + bias_mat)

        x = torch.bmm(w, torch.transpose(x, 1, 2))
        #print("x after bmm", x.data.shape)

        x = torch.transpose(x, 1, 2)

        #x = x + old
        x = torch.cat((x, old), dim=1)
        x = torch.mul(x, unpacked_masks)

        x = self.bn4(x)
        x = self.elu(x)
        x = torch.mul(x, unpacked_masks)
        x = torch.transpose(x, 1, 2)

        x = self.dropout2(x)

        x = self.fc(x)

        #print("x after fc", x.data.shape)

        return x

In [8]:
from __future__ import print_function

import numpy as np
# np.set_printoptions(threshold=np.nan)
from torch.autograd import Variable
import torch
import torch.nn as nn
torch.set_printoptions(threshold=50000)
import torch.optim as optim
from torch import squeeze
from torch.optim import lr_scheduler
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
from torch import index_select
from sklearn.metrics import confusion_matrix, roc_auc_score, matthews_corrcoef

In [9]:
def get_variables(dist_matx_path_heavy,dist_matx_path_light,cdr_loop,cut_off):
    """
    For every CDR get the corresponding seq and check if dist is less than cutoff
    and substitute 1 orelse 0 
    
    E.g. - CDR - TCRASGNIHNYLAWY
           Seq.no - ['22','23','24','25','26','27','28','29','30','31','32','33','34','35','36']
           
           Output - [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
    """
    pdb_name = cdr_loop['PDB_Name'].to_numpy()
    seq_no = cdr_loop['seq_no'].to_numpy()
    chain_type = cdr_loop['Type'].to_numpy()

    all_var = []
    for i in range(len(pdb_name)):
        if chain_type[i] == 'H1' or chain_type[i] == 'H2' or chain_type[i] == 'H3':
            dist = pd.read_pickle(dist_matx_path_heavy+pdb_name[i]+"heavy"+".pkl")
            var = []
            for j in range(len(seq_no[i])):
                first_index = dist.index.get_level_values(0)[0] ##get the first index for slicing
                row_slc = dist.loc[(first_index, seq_no[i][j])] ##slice multi index for specific row

                if np.any(row_slc < cut_off) == True:
                    var.append(1)
                else:
                    var.append(0)
            all_var.append(var)

        elif chain_type[i] == 'L1' or chain_type[i] == 'L2' or chain_type[i] == 'L3':
            dist = pd.read_pickle(dist_matx_path_light+pdb_name[i]+"light"+".pkl")
            var = []
            for j in range(len(seq_no[i])):
                first_index = dist.index.get_level_values(0)[0] ##get the first index for slicing
                row_slc = dist.loc[(first_index, seq_no[i][j])] ##slice multi index for specific row

                if np.any(row_slc < cut_off) == True:
                    var.append(1)
                else:
                    var.append(0)
            all_var.append(var)
    
    return all_var

In [10]:
def flatten_with_lengths(matrix, lengths):
    seqs = []
    for i, example in enumerate(matrix):
        seq = example[:lengths[i]]
        seqs.append(seq)
    return np.concatenate(seqs)

def permute_training_data(cdrs, masks, lengths, lbls):
    """
    Shuffle training data
    :param cdrs:
    :param masks:
    :param lengths:
    :param lbls:
    :return:
    """
    index = torch.randperm(cdrs.shape[0])
    if use_cuda:
        index = index.cuda()

    cdrs = torch.index_select(cdrs, 0, index)
    lbls = torch.index_select(lbls, 0, index)
    masks = torch.index_select(masks, 0, index)
    lengths = [lengths[i] for i in index]

    return cdrs, masks, lengths, lbls

def sort_batch(cdrs, masks, lengths, lbls):
    """
    Sort antibody amino acid sequences by length -- for RNN
    :param cdrs:
    :param masks:
    :param lengths:
    :param lbls:
    :return: sorted input
    """
    order = np.argsort(lengths)
    order = order.tolist()
    order.reverse()
    lengths.sort(reverse=True)
    index = Variable(torch.LongTensor(order))
    if use_cuda:
        index = index.cuda()

    cdrs = torch.index_select(cdrs, 0, index)
    lbls = torch.index_select(lbls, 0, index)
    masks = torch.index_select(masks, 0, index)
    return cdrs, masks, lengths, lbls

In [28]:
def atrous_self_run(cdrs_train, lbls_train, masks_train, lengths_train, weights_template, weights_template_number,
               cdrs_test, lbls_test, masks_test, lengths_test):
    """
    :param cdrs_train: antibody amino acid training sequences
    :param lbls_train:
    :param masks_train:
    :param lengths_train:
    :param weights_template:
    :param weights_template_number:
    :param cdrs_test:
    :param lbls_test:
    :param masks_test:
    :param lengths_test:
    :return: binding probabilities for testing antibody amino acids
    """

    # print("atrous self run", file=print_file)
    print("atrous self run")
    model = AtrousSelf()

    ignored_params = list(map(id, [model.conv1.weight, model.conv2.weight, model.conv3.weight,
                                   model.aconv1.weight, model.aconv2.weight]))
    base_params = filter(lambda p: id(p) not in ignored_params,
                         model.parameters())

    optimizer = optim.Adam([
        {'params': base_params},
        {'params': model.conv1.weight, 'weight_decay': 0.01},
        {'params': model.conv2.weight, 'weight_decay': 0.01},
        {'params': model.conv3.weight, 'weight_decay': 0.01},
        {'params': model.aconv1.weight, 'weight_decay': 0.01},
        {'params': model.aconv2.weight, 'weight_decay': 0.01}
    ], lr=0.01)

    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10], gamma=0.1)

    total_input = cdrs_train
    total_lbls = lbls_train
    total_masks = masks_train
    total_lengths = lengths_train

    if use_cuda:
        print("using cuda")
        model.cuda()
        total_input = total_input.cuda()
        total_lbls = total_lbls.cuda()
        total_masks = total_masks.cuda()
        cdrs_test = cdrs_test.cuda()
        lbls_test = lbls_test.cuda()
        masks_test = masks_test.cuda()

    times = []

    for epoch in range(epochs):
        model.train(True)
        scheduler.step()
        epoch_loss = 0

        batches_done=0

        total_input, total_masks, total_lengths, total_lbls = \
            permute_training_data(total_input, total_masks, total_lengths, total_lbls)


        total_time = 0


        for j in range(0, cdrs_train.shape[0], batch_size):
            batches_done +=1
            interval = [x for x in range(j, min(cdrs_train.shape[0], j + batch_size))]
            interval = torch.LongTensor(interval)
            if use_cuda:
                interval = interval.cuda()

            input = Variable(index_select(total_input, 0, interval), requires_grad=True)
            masks = Variable(index_select(total_masks, 0, interval))
            lengths = total_lengths[j:j + batch_size]
            lbls = Variable(index_select(total_lbls, 0, interval))

            output = model(input, masks)

            loss_weights = (lbls * 1.5 + 1) * masks
            max_val = (-output).clamp(min=0)
            loss = loss_weights * (output - output * lbls + max_val + ((-max_val).exp() + (-output - max_val).exp()).log())
            masks_added = masks.sum()
            loss = loss.sum() / masks_added

            #print("Epoch %d - Batch %d has loss %d " % (epoch, j, loss.data), file=monitoring_file)
            epoch_loss +=loss

            model.zero_grad()

            start = time.time()

            loss.backward()
            optimizer.step()

            total_time += time.time() - start

        print("Epoch %d - loss is %f : " % (epoch, epoch_loss.data/batches_done))
        # print("--- %s seconds ---" % (total_time))
        times.append(total_time)

        model.eval()

        cdrs_test2, masks_test2, lengths_test2, lbls_test2 = sort_batch(cdrs_test, masks_test, list(lengths_test),
                                                                    lbls_test)

        probs_test2 = model(cdrs_test2, masks_test2)

        # K.mean(K.equal(lbls_test, K.round(y_pred)), axis=-1)

        sigmoid = nn.Sigmoid()
        probs_test2 = sigmoid(probs_test2)

        probs_test2 = probs_test2.data.cpu().numpy().astype('float32')
        lbls_test2 = lbls_test2.data.cpu().numpy().astype('int32')

        probs_test2 = flatten_with_lengths(probs_test2, lengths_test2)
        lbls_test2 = flatten_with_lengths(lbls_test2, lengths_test2)

        print("Roc", roc_auc_score(lbls_test2, probs_test2))

    torch.save(model.state_dict(), weights_template.format(weights_template_number))

    times_mean = np.mean(times)
    times_std = 2 * np.std(times)

    print("Time mean", times_mean)
    print("Time std", times_std)

    # print("test", file=track_f)
    print("test")
    model.eval()

    probs_test = model(cdrs_test, masks_test)

    # K.mean(K.equal(lbls_test, K.round(y_pred)), axis=-1)

    sigmoid = nn.Sigmoid()
    probs_test = sigmoid(probs_test)

    probs_test1 = probs_test.data.cpu().numpy().astype('float32')
    lbls_test1 = lbls_test.data.cpu().numpy().astype('int32')

    probs_test1 = flatten_with_lengths(probs_test1, list(lengths_test))
    lbls_test1 = flatten_with_lengths(lbls_test1, list(lengths_test))

    print("Roc", roc_auc_score(lbls_test1, probs_test1))

    return probs_test, lbls_test, probs_test1, lbls_test1  # get them in kfold, append, concatenate do roc on them

In [12]:
def kfold_cv_eval(cdrs, lbls, masks, lengths, ag, ag_masks, ag_lengths, output_file="crossval-data.p",
                  weights_template="weights-fold-{}.h5", seed=0):
    """
    Performs 10-fold cross-vallidation
    :param dataset: contains antibody amino acids, ground truth values, antigen atoms
    :param output_file: where to print weights
    :param weights_template:
    :param seed: cv
    :return:
    """
    # cdrs, lbls, masks, lengths, ag, ag_masks, ag_lengths, dist_mat = \
    #     dataset["cdrs"], dataset["lbls"], dataset["masks"], dataset["lengths"],\
    #     dataset["ag"], dataset["ag_masks"], dataset["ag_lengths"], dataset["dist_mat"]


    print("cdrs", cdrs.shape)
    print("ag", ag.shape)
    #print("lbls", lbls, file=data_file)
    #print("masks", masks, file=data_file)
    #print("lengths", lengths, file=data_file)

    kf = KFold(n_splits=NUM_SPLIT, random_state=seed, shuffle=True)

    all_lbls2 = []
    all_probs2 = []
    all_masks = []

    all_probs1 = []
    all_lbls1 = []

    for i, (train_idx, test_idx) in enumerate(kf.split(cdrs)):
        print("Fold: ", i + 1)
        #print(train_idx, )
        #print(test_idx)

        lengths_train = [lengths[i] for i in train_idx]
        lengths_test = [lengths[i] for i in test_idx]

        ag_lengths_train = [ag_lengths[i] for i in train_idx]
        ag_lengths_test = [ag_lengths[i] for i in test_idx]

        #print("train_idx", train_idx)

        print("len(train_idx",len(train_idx))

        train_idx = torch.from_numpy(train_idx)
        test_idx = torch.from_numpy(test_idx)

        cdrs_train = index_select(cdrs, 0, train_idx)
        lbls_train = index_select(lbls, 0, train_idx)
        mask_train = index_select(masks, 0, train_idx)
        ag_train = index_select(ag, 0, train_idx)
        ag_masks_train = index_select(ag_masks, 0, train_idx)
        # dist_mat_train = index_select(dist_mat, 0, train_idx)

        cdrs_test = Variable(index_select(cdrs, 0, test_idx))
        lbls_test = Variable(index_select(lbls, 0, test_idx))
        mask_test = Variable(index_select(masks, 0, test_idx))
        ag_test = Variable(index_select(ag, 0, test_idx))
        ag_masks_test = Variable(index_select(ag_masks, 0, test_idx))
        # dist_mat_test = Variable(index_select(dist_mat, 0, test_idx))

        code = 5
        if code ==1:
            probs_test1, lbls_test1, probs_test2, lbls_test2 = \
                simple_run(cdrs_train, lbls_train, mask_train, lengths_train, weights_template, i,
                                    cdrs_test, lbls_test, mask_test, lengths_test)
        if code == 2:
            probs_test1, lbls_test1, probs_test2, lbls_test2 = \
                attention_run(cdrs_train, lbls_train, mask_train, lengths_train, weights_template, i,
                              cdrs_test, lbls_test, mask_test, lengths_test)

        if code == 3:
            probs_test1, lbls_test1, probs_test2, lbls_test2 = \
                atrous_run(cdrs_train, lbls_train, mask_train, lengths_train, weights_template, i,
                                     cdrs_test, lbls_test, mask_test, lengths_test)

        if code == 4:
            probs_test1, lbls_test1, probs_test2, lbls_test2 = \
                antigen_run(cdrs_train, lbls_train, mask_train, lengths_train,
                            ag_train, ag_masks_train, ag_lengths_train, weights_template, i,
                           cdrs_test, lbls_test, mask_test, lengths_test,
                            ag_test, ag_masks_test, ag_lengths_test)

        if code == 5:
            probs_test1, lbls_test1, probs_test2, lbls_test2 = \
                atrous_self_run(cdrs_train, lbls_train, mask_train, lengths_train, weights_template, i,
                           cdrs_test, lbls_test, mask_test, lengths_test)

        if code == 6:
            probs_test1, lbls_test1, probs_test2, lbls_test2 = \
                rnn_run(cdrs_train, lbls_train, mask_train, lengths_train, weights_template, i,
                                    cdrs_test, lbls_test, mask_test, lengths_test)

        if code == 7:
            probs_test1, lbls_test1, probs_test2, lbls_test2 = \
                xself_run(cdrs_train, lbls_train, mask_train, lengths_train,
                            ag_train, ag_masks_train, ag_lengths_train, dist_mat_train, weights_template, i,
                            cdrs_test, lbls_test, mask_test, lengths_test,
                            ag_test, ag_masks_test, ag_lengths_test, dist_mat_test)

        # print("test", file=track_f)
        print("test")

        lbls_test2 = np.squeeze(lbls_test2)
        all_lbls2 = np.concatenate((all_lbls2, lbls_test2))
        all_lbls1.append(lbls_test1)

        probs_test_pad = torch.zeros(probs_test1.data.shape[0], MAX_CDR_LENGTH, probs_test1.data.shape[2])
        probs_test_pad[:probs_test1.data.shape[0], :probs_test1.data.shape[1], :] = probs_test1.data
        probs_test_pad = Variable(probs_test_pad)

        probs_test2 = np.squeeze(probs_test2)
        #print(probs_test)
        all_probs2 = np.concatenate((all_probs2, probs_test2))
        #print(all_probs)
        #print(type(all_probs))

        all_probs1.append(probs_test_pad)

        all_masks.append(mask_test)

    lbl_mat1 = torch.cat(all_lbls1)
    prob_mat1 = torch.cat(all_probs1)
    #print("end", all_probs)
    mask_mat = torch.cat(all_masks)

    with open(output_file, "wb") as f:
        pickle.dump((lbl_mat1, prob_mat1, mask_mat, all_lbls2, all_probs2), f)

 **Building the input Matrices**

In [13]:
summary_files = pd.read_pickle("/content/drive/My Drive/Peritia_Fast-Parapred/summary_file_cleaned.pkl")
summary_files = summary_files.drop(summary_files[summary_files.pdb.isin(["4ydl","4ydv","5ig7","5ies","4hii","5w0d","5i9q","4hkx",
                                                          "5te4","5te6","4xvs","4xvt","3gbm","2xqb","5if0","4s1q",
                                                          "4xmp","4xny","3mlt","6mvl","4xh2","5ifj","5te7","5occ",
                                                          "4dqo","4lsq","4lsp","4lsr","4yaq","3h3p","3idi",
                                                          "3lrs","3qnz","5x08","5alc","2p8m","6pbw","5csz",
                                                          "6bkb","5ado","4xaw","5wnb","5dt1","4ob5","4xbe",
                                                          "5fgb","4y5y","3u4e","4xcf","4rnr","3drq","6uyf",
                                                          "4m8q","3d0l","2jb6","3u2s","3lhp","3mug","1bvk","5e08","5gkr",
                                                                        "6mwn","6u8k","6u8d","6df1"])].index)

pdb_names = summary_files['pdb'].to_numpy()
Hchain_id = summary_files['Hchain'].to_numpy()
Lchain_id = summary_files['Lchain'].to_numpy()
Achain_id = summary_files['antigen_chain_list'].to_numpy()
 
epi_res = []
epi_res_num = []
ext_epi_res_num = []
ext_epi_res = []
A_pdb = []
# for i in range(len(pdb_names)):
#     epi_residues,epi_res_number,ext_epi_res_number, ext_epi_residues = ext_epitope_sorted(Hchain_id[i],Lchain_id[i],Achain_id[i],pdb_names[i])
#     A_pdb.append(pdb_names[i])
#     epi_res.append(epi_residues)
#     epi_res_num.append(epi_res_number)
#     ext_epi_res_num.append(ext_epi_res_number)
#     ext_epi_res.append(ext_epi_residues)

In [14]:
 Sorted_epi = pd.read_pickle("/content/drive/My Drive/Peritia_Fast-Parapred/Sorted_Antigen_epitope_dataframe.pkl")
 Sorted_epi.head()

,pdb,sorted_epi_residues,sorted_epi_res_number,sorted_ext_epi_res_number,sorted_ext_epi_residues
0,4ydk,"[Q, L, T, G, G, S, T, E, N, A, K, T, Q, P, S, ...","[105 , 122 , 123 , 124 , 198 , 199 , 257 , 275...","[52 , 54 , 65 , 66 , 69 , 93 , 94 , 95 , 96 , ...","[L, C, V, H, W, F, N, M, W, K, N, N, M, V, E, ..."
1,4ydj,"[W, K, E, E, N, N, A, K, T, I, R, P, S, G, G, ...","[96 , 97 , 102 , 275 , 279 , 280 , 281 , 282 ,...","[47 , 48 , 49 , 50 , 51 , 52 , 69 , 92 , 93 , ...","[D, A, D, T, T, L, W, N, F, N, M, W, K, N, N, ..."
2,1u8i,"[E, L, D, K, W, A, N]","[1 , 2 , 3 , 4 , 5 , 6 , 7 ]","[1 , 2 , 3 , 4 , 5 , 6 , 7 ]","[E, L, D, K, W, A, N]"
3,1u8l,"[D, L, D, R, W, A, S]","[1 , 2 , 3 , 4 , 5 , 6 , 7 ]","[1 , 2 , 3 , 4 , 5 , 6 , 7 ]","[D, L, D, R, W, A, S]"
4,5y9j,"[K, G, S, Y, T, Y, A, M, G, H, L, Q, K, V, G, ...","[160 , 161 , 162 , 163 , 205 , 206 , 207 , 208...","[156 , 157 , 158 , 159 , 160 , 161 , 162 , 163...","[P, T, I, Q, K, G, S, Y, T, F, V, W, E, N, K, ..."


In [15]:
epi_res = Sorted_epi['sorted_epi_residues'].to_numpy()
epi_res_num = Sorted_epi['sorted_epi_res_number'].to_numpy()
ext_epi_res_num = Sorted_epi['sorted_ext_epi_res_number'].to_numpy()
ext_epi_res = Sorted_epi['sorted_ext_epi_residues'].to_numpy()

In [16]:
##Joining all the residue abbrevations for making a sequence and repeating each seq 6 times/pdb

Anti_str = []
for i in range(len(ext_epi_res)):
#     print(i)
    str1 = ''.join(ext_epi_res[i])
#     print(repeat(str1, 6))
#     print(str1,i,len(A_mat[i]))
    Anti_str.append(str1)

import itertools
Anti_seq = list(itertools.chain.from_iterable(itertools.repeat(x, 6) for x in Anti_str))

MAX_EXT_AG_LENGTH = maxi_len(Anti_seq)
MAX_EXT_AG_LENGTH

288

In [17]:
## Generating Labels for Extended Epitope Antigen Matrix

labels = []
for i in range(len(ext_epi_res_num)):
    label = []
    for j in ext_epi_res_num[i]:
#     print(i)
        if j in epi_res_num[i]:
            label.append(1)
        else:
            label.append(0)
    labels.append(label)

import itertools
Anti_repeat_labels = list(itertools.chain.from_iterable(itertools.repeat(x, 6) for x in labels))

# Antigen_lbls = np.stack(labels)
MAX_EXT_AG_LENGTH = 288

label_mats = []
for i in range(len(Anti_repeat_labels)):
    label_mat = torch.tensor(Anti_repeat_labels[i])
    label_mat_pad = torch.zeros((MAX_EXT_AG_LENGTH, 1))
    label_mat_pad[:label_mat.shape[0], 0] = label_mat
    label_mats.append(label_mat_pad)

ext_epi_lbls = torch.stack(label_mats)
ext_epi_lbls.shape

torch.Size([3774, 288, 1])

In [18]:
NUM_FEATURES = 28
MAX_EXT_AG_LENGTH = 288
# cdr_mats = []
ext_epi_mats = []

ext_epi_lengths = []
for i in range(len(Anti_seq)):
#     on_hot = seq_to_one_hot(df['CDR'][i])
#     print(i)
    ext_epi_mat = seq_to_one_hot(Anti_seq[i])
    ext_epi_mat_pad = torch.zeros((MAX_EXT_AG_LENGTH, NUM_FEATURES))
    ext_epi_mat_pad[:ext_epi_mat.shape[0], :] = ext_epi_mat
    ext_epi_mats.append(ext_epi_mat_pad)
    ext_epi_lengths.append(ext_epi_mat.shape[0])

ext_epi = torch.stack(ext_epi_mats)
ext_epi.shape


torch.Size([3774, 288, 28])

In [19]:
ext_epi_masks = []
for i in range(len(Anti_seq)):
    ext_epi_mask = torch.zeros((MAX_EXT_AG_LENGTH, 1), dtype=int)
    ext_epi_mask[:len(Anti_seq[i]), 0] = 1
    ext_epi_masks.append(ext_epi_mask)
ext_epi_masks = torch.stack(ext_epi_masks)
ext_epi_masks.shape

torch.Size([3774, 288, 1])

In [20]:
NUM_FEATURES = 28+6
Max_len_CDR = 38
cdr_mats = []

cdr_loop = pd.read_pickle("/content/drive/My Drive/Peritia_Fast-Parapred/CDRs_seqno_pdb.pkl")
cdr_loop = cdr_loop.drop(cdr_loop[cdr_loop.PDB_Name.isin(["4ydl","4ydv","5ig7","5ies","4hii","5w0d","5i9q","4hkx",
                                                          "5te4","5te6","4xvs","4xvt","3gbm","2xqb","5if0","4s1q",
                                                          "4xmp","4xny","3mlt","6mvl","4xh2","5ifj","5te7","5occ",
                                                          "4dqo","4lsq","4lsp","4lsr","4yaq","3h3p","3idi",
                                                          "3lrs","3qnz","5x08","5alc","2p8m","6pbw","5csz",
                                                          "6bkb","5ado","4xaw","5wnb","5dt1","4ob5","4xbe",
                                                          "5fgb","4y5y","3u4e","4xcf","4rnr","3drq","6uyf",
                                                          "4m8q","3d0l","2jb6","3u2s","3lhp","3mug","1bvk","5e08","5gkr",
                                                                        "6mwn","6u8k","6u8d","6df1"])].index)
cdr_loop.reset_index(drop=True, inplace=True)

for i in range(len(cdr_loop['CDR'])):
#     on_hot = seq_to_one_hot(df['CDR'][i])
    cdr_mat = cdrseq_to_one_hot(cdr_loop['CDR'][i],i)
    cdr_mat_pad = torch.zeros((Max_len_CDR, NUM_FEATURES))
    cdr_mat_pad[:cdr_mat.shape[0], :] = cdr_mat
    cdr_mats.append(cdr_mat_pad)
cdrs_seq = torch.stack(cdr_mats)
cdrs_seq.shape

torch.Size([3774, 38, 34])

In [21]:
all_cdrs_lengths = []
for i in range(len(cdr_loop['CDR'])):
#     on_hot = seq_to_one_hot(df['CDR'][i])
    cdr_mat = cdrseq_to_one_hot(cdr_loop['CDR'][i],i)
    cdr_mat_pad = torch.zeros((Max_len_CDR, NUM_FEATURES))
    cdr_mat_pad[:cdr_mat.shape[0], :] = cdr_mat
    cdr_mats.append(cdr_mat_pad)
    all_cdrs_lengths.append(cdr_mat.shape[0])
max(all_cdrs_lengths)

28

In [22]:
Max_len_CDR = 38

cdr_masks = []
for i in range(len(cdr_loop['CDR'])):
    cdr_mask = torch.zeros((Max_len_CDR, 1), dtype=int)
    cdr_mask[:len(cdr_loop['CDR'][i]), 0] = 1
    cdr_masks.append(cdr_mask)
cdr_mask = torch.stack(cdr_masks)
cdr_mask.shape

torch.Size([3774, 38, 1])

In [23]:
cdr_loop = pd.read_pickle("/content/drive/My Drive/Peritia_Fast-Parapred/CDRs_seqno_pdb.pkl")
cdr_loop = cdr_loop.drop(cdr_loop[cdr_loop.PDB_Name.isin(["4ydl","4ydv","5ig7","5ies","4hii","5w0d","5i9q","4hkx",
                                                          "5te4","5te6","4xvs","4xvt","3gbm","2xqb","5if0","4s1q",
                                                          "4xmp","4xny","3mlt","6mvl","4xh2","5ifj","5te7","5occ",
                                                          "4dqo","4lsq","4lsp","4lsr","4yaq","3h3p","3idi",
                                                          "3lrs","3qnz","5x08","5alc","2p8m","6pbw","5csz",
                                                          "6bkb","5ado","4xaw","5wnb","5dt1","4ob5","4xbe",
                                                          "5fgb","4y5y","3u4e","4xcf","4rnr","3drq","6uyf",
                                                          "4m8q","3d0l","2jb6","3u2s","3lhp","3mug","1bvk",
                                                          "5e08","5gkr","6mwn","6u8k","6u8d","6df1"])].index)
cdr_loop.reset_index(drop=True, inplace=True)

dist_matx_path_light = "/content/drive/My Drive/Peritia_Fast-Parapred/Light_chain_dist_clean/"
dist_matx_path_heavy = "/content/drive/My Drive/Peritia_Fast-Parapred/Heavy_chain_dist_clean/"
cut_off = 5
# all_variables = get_variables(dist_matx_path_heavy,dist_matx_path_light,cdr_loop,cut_off)
with open('/content/drive/My Drive/Peritia_Fast-Parapred/all_variables.pkl', 'rb') as f:
    all_variables = pickle.load(f)
# all_variables


In [24]:
cont_mats = []
cdr_lengths = []
Max_len_CDR = 38
"""
Pad the variables to Max CDR len with 0
Here max len of CDR is 38

E.g - Input = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
    Output = [0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[1.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.]

"""
for i in range(len(all_variables)):
    cont_mat = torch.tensor(all_variables[i])
    cont_mat_pad = torch.zeros((Max_len_CDR, 1))
    cont_mat_pad[:cont_mat.shape[0], 0] = cont_mat
    cont_mats.append(cont_mat_pad)
    cdr_lengths.append(cont_mat.shape[0])


cdr_lbls = torch.stack(cont_mats)

cdr_lbls.shape
# cdr_lengths

torch.Size([3774, 38, 1])

In [25]:
MAX_CDR_LENGTH = 38

AG_NUM_FEATURES = 28

NUM_FEATURES = 34

MAX_EXT_AG_LENGTH = 288

use_cuda = torch.cuda.is_available()

epochs = 20

batch_size = 32

In [26]:
NUM_ITERATIONS = 10
NUM_SPLIT = 10
output_file="crossval-data.p",
# weights_template="weights-fold-{}.h5"
import os 
def run_cv(output_folder="cv-only_ab-seq",
           num_iters=NUM_ITERATIONS):
#     cache_file = dataset.split("/")[-1] + ".p"
#     dataset = open_dataset(dataset_cache=cache_file)

    dir =  output_folder + "/weights"
    if not os.path.exists(dir):
        os.makedirs(output_folder + "/weights")
    for i in range(num_iters):
        #i=0
        print("Crossvalidation run", i+1)
        output_file = "{}/run-{}.p".format(output_folder, i)
        weights_template = output_folder + "/weights/run-" + \
                           str(i) + "-fold-{}.pth.tar"
        kfold_cv_eval(cdrs_seq, cdr_lbls, cdr_mask, all_cdrs_lengths, ext_epi, ext_epi_masks, ext_epi_lengths, output_file,
                  weights_template, seed=0)

In [29]:
import time
import os
start_time = time.time()
run_cv(output_folder="/content/drive/My Drive/Peritia_Fast-Parapred/cv-only_ab-seq",
           num_iters=NUM_ITERATIONS)
end_time = time.time() - start_time
hours, rem = divmod(end_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

Crossvalidation run 1
cdrs torch.Size([3774, 38, 34])
ag torch.Size([3774, 288, 28])
Fold:  1
len(train_idx 3396
atrous self run
using cuda


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 0 - loss is 0.999564 : 
Roc 0.7857975192924458
Epoch 1 - loss is 0.795327 : 
Roc 0.7597767036750398
Epoch 2 - loss is 0.814754 : 
Roc 0.809584037989318
Epoch 3 - loss is 0.801573 : 
Roc 0.798448184296602
Epoch 4 - loss is 0.784410 : 
Roc 0.7966553921709137
Epoch 5 - loss is 0.771625 : 
Roc 0.8587825614762582
Epoch 6 - loss is 0.767178 : 
Roc 0.8238985072440227
Epoch 7 - loss is 0.768881 : 
Roc 0.8377930692107535
Epoch 8 - loss is 0.765176 : 
Roc 0.8301629785689034
Epoch 9 - loss is 0.713829 : 
Roc 0.8676908861307778
Epoch 10 - loss is 0.694774 : 
Roc 0.8779388533774122
Epoch 11 - loss is 0.682939 : 
Roc 0.865611752865601
Epoch 12 - loss is 0.677310 : 
Roc 0.8802940764308896
Epoch 13 - loss is 0.668684 : 
Roc 0.8881450189978438
Epoch 14 - loss is 0.667091 : 
Roc 0.887791297618024
Epoch 15 - loss is 0.661821 : 
Roc 0.8851972081291025
Epoch 16 - loss is 0.667963 : 
Roc 0.8889017279597939
Epoch 17 - loss is 0.653037 : 
Roc 0.8943533567103952
Epoch 18 - loss is 0.655542 : 
Roc 0.89450

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 0 - loss is 0.970274 : 
Roc 0.7791640458524813
Epoch 1 - loss is 0.803238 : 
Roc 0.8074058592847423
Epoch 2 - loss is 0.785385 : 
Roc 0.7852020808194913
Epoch 3 - loss is 0.778322 : 
Roc 0.7841203932504942
Epoch 4 - loss is 0.769521 : 
Roc 0.8028149550075736
Epoch 5 - loss is 0.764313 : 
Roc 0.7704949006443994
Epoch 6 - loss is 0.764938 : 
Roc 0.8140847234345716
Epoch 7 - loss is 0.752013 : 
Roc 0.8106866238992579
Epoch 8 - loss is 0.762198 : 
Roc 0.8140127179019795
Epoch 9 - loss is 0.711384 : 
Roc 0.8441986807142308
Epoch 10 - loss is 0.688822 : 
Roc 0.8501019309517086
Epoch 11 - loss is 0.674427 : 
Roc 0.8478670517124078
Epoch 12 - loss is 0.669260 : 
Roc 0.8551401119355087
Epoch 13 - loss is 0.661518 : 
Roc 0.8569749492952684
Epoch 14 - loss is 0.660742 : 
Roc 0.8530764213370645
Epoch 15 - loss is 0.655257 : 
Roc 0.8618937896331288
Epoch 16 - loss is 0.643018 : 
Roc 0.8641018924417858
Epoch 17 - loss is 0.635906 : 
Roc 0.8626074266385972
Epoch 18 - loss is 0.641835 : 
Roc 0.8

...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 0 - loss is 0.970502 : 
Roc 0.7668056552822178
Epoch 1 - loss is 0.818774 : 
Roc 0.7897647274600399
Epoch 2 - loss is 0.811244 : 
Roc 0.7988708557067933
Epoch 3 - loss is 0.788011 : 
Roc 0.8355265827922077
Epoch 4 - loss is 0.774616 : 
Roc 0.8185085422390108
Epoch 5 - loss is 0.756546 : 
Roc 0.8127368725024975
Epoch 6 - loss is 0.758355 : 
Roc 0.8168794096528472
Epoch 7 - loss is 0.756930 : 
Roc 0.8154698816808192
Epoch 8 - loss is 0.762142 : 
Roc 0.8316622830294705
Epoch 9 - loss is 0.707129 : 
Roc 0.8617140671828172
Epoch 10 - loss is 0.685604 : 
Roc 0.8629257461288711
Epoch 11 - loss is 0.680610 : 
Roc 0.8690694071553446
Epoch 12 - loss is 0.674323 : 
Roc 0.8707614650974027
Epoch 13 - loss is 0.665680 : 
Roc 0.868258206637113
Epoch 14 - loss is 0.663047 : 
Roc 0.8770889266983016
Epoch 15 - loss is 0.661068 : 
Roc 0.8749367819680319
Epoch 16 - loss is 0.657552 : 
Roc 0.8732864791458542
Epoch 17 - loss is 0.646020 : 
Roc 0.8780303680694306
Epoch 18 - loss is 0.652922 : 
Roc 0.87

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 0 - loss is 0.998824 : 
Roc 0.5563065510652658
Epoch 1 - loss is 0.806770 : 
Roc 0.8307460004388556
Epoch 2 - loss is 0.789999 : 
Roc 0.7982023990496736
Epoch 3 - loss is 0.790896 : 
Roc 0.8096109798846735
Epoch 4 - loss is 0.776204 : 
Roc 0.7996986302624681
Epoch 5 - loss is 0.760136 : 
Roc 0.8029785972525305
Epoch 6 - loss is 0.774840 : 
Roc 0.7977762427546142
Epoch 7 - loss is 0.753243 : 
Roc 0.8021361734870411
Epoch 8 - loss is 0.758107 : 
Roc 0.8163593219972012
Epoch 9 - loss is 0.715072 : 
Roc 0.8578576184129499
Epoch 10 - loss is 0.687794 : 
Roc 0.862426463577273
Epoch 11 - loss is 0.682654 : 
Roc 0.8708454965876268
Epoch 12 - loss is 0.671242 : 
Roc 0.8665772716920216
Epoch 13 - loss is 0.661918 : 
Roc 0.8730445629993077
Epoch 14 - loss is 0.651040 : 
Roc 0.8741510704288336
Epoch 15 - loss is 0.650311 : 
Roc 0.8742674462807818
Epoch 16 - loss is 0.645179 : 
Roc 0.8612358490919562
Epoch 17 - loss is 0.638750 : 
Roc 0.879656959796411
Epoch 18 - loss is 0.635463 : 
Roc 0.872

/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [30]:
def open_crossval_results(folder="/content/drive/My Drive/Peritia_Fast-Parapred/cv-only_ab-seq", num_results=NUM_ITERATIONS,
                          loop_filter=None, flatten_by_lengths=True):
    class_probabilities = []
    labels = []
    class_probabilities1 = []
    labels1 = []
    for r in range(num_results):
        result_filename = "{}/run-{}.p".format(folder, r)
        with open(result_filename, "rb") as f:
            lbl_mat, prob_mat, mask_mat, all_lbls, all_probs = pickle.load(f)
            lbl_mat = lbl_mat.data.cpu().numpy()
            prob_mat = prob_mat.data.cpu().numpy()
            mask_mat = mask_mat.data.cpu().numpy()
        # Get entries corresponding to the given loop
        if loop_filter is not None:
            lbl_mat = lbl_mat[loop_filter::6]
            prob_mat = prob_mat[loop_filter::6]
            mask_mat = mask_mat[loop_filter::6]
        """""
        if not flatten_by_lengths:
            class_probabilities.append(prob_mat)
            labels.append(lbl_mat)
            continue
        """
        # Discard sequence padding
        seq_lens = np.sum(np.squeeze(mask_mat), axis=1)
        seq_lens = seq_lens.astype(int)
        p = flatten_with_lengths(prob_mat, seq_lens)
        l = flatten_with_lengths(lbl_mat, seq_lens)
        class_probabilities.append(p)
        labels.append(l)
        #class_probabilities1 = np.concatenate((class_probabilities1, all_probs))
        #labels1 = np.concatenate((labels1,all_lbls))
        class_probabilities1.append(all_probs)
        labels1.append(all_lbls)
    return labels, class_probabilities, labels1, class_probabilities1

In [31]:
def process_cv_results():
    """
    Plots PR curves, output computed metrics
    :return:void
    """
    import matplotlib.pyplot as plt
    plt.rcParams["font.family"] = "Times New Roman"

    # Plot ROC per loop type
    fig = None
    cols = [("#D6083B", "#EB99A9"),
            ("#0072CF", "#68ACE5"),
            ("#EA7125", "#F3BD48"),
            ("#55A51C", "#AAB300"),
            ("#8F2BBC", "#AF95A3"),
            ("#00B1C1", "#91B9A4")]


    # Plot PR curves
    print("Plotting PR curves")
    labels, probs, labels1, probs1 = open_crossval_results("/content/drive/My Drive/Peritia_Fast-Parapred/cv-only_ab-seq", NUM_ITERATIONS)

    #labels, probs = initial_open_crossval_results("parapred-cv-ab-seq", NUM_ITERATIONS)
    #selflabels, selfprobs, selflabels1, selfprobs1 = open_crossval_results("self-cv-ab-seq", NUM_ITERATIONS)
    #_,_,aglabels, agprobs = open_crossval_results("ag-cv-ab-seq", NUM_ITERATIONS)


    fig1 = plot_pr_curve(labels1, probs1, colours=("#0072CF", "#68ACE5"),label="Parapred")

    #fig1 = plot_abip_pr(fig1)
    #fig1 = plot_pr_curve(selflabels1, selfprobs1, colours=("#228B18", "#006400"), label="Fast-Parapred", plot_fig=fig1)
    #fig1 = plot_pr_curve(aglabels, agprobs, colours=("#FF8C00", "#FFA500"), label="AG-Fast-Parapred", plot_fig=fig1)
    #fig1.savefig("pr1.pdf")

    print("Printing PDB for visualisation")
    if visualisation_flag:
        print_probabilities()

    # Computing overall classifier metrics
    print("Computing classifier metrics")
    initial_compute_classifier_metrics(labels, probs, threshold=0.4913739)

#run_cv()
import matplotlib.pyplot as plt

In [32]:
labels, class_probabilities, labels1, class_probabilities1 = open_crossval_results(folder="/content/drive/My Drive/Peritia_Fast-Parapred/cv-only_ab-seq", num_results=NUM_ITERATIONS,
                          loop_filter=None, flatten_by_lengths=True)

In [33]:
def helper_compute_metrics(matrices, aucs, mcorrs):
    matrices = np.stack(matrices)
    mean_conf = np.mean(matrices, axis=0)
    errs_conf = 2 * np.std(matrices, axis=0)

    tps = matrices[:, 1, 1]
    fns = matrices[:, 1, 0]
    fps = matrices[:, 0, 1]

    tpsf = tps.astype(float)
    fnsf = fns.astype(float)
    fpsf = fps.astype(float)

    recalls = tpsf / (tpsf + fnsf)
    precisions = tpsf / (tpsf + fpsf)

    rec = np.mean(recalls)
    rec_err = 2 * np.std(recalls)
    prec = np.mean(precisions)
    prec_err = 2 * np.std(precisions)

    fscores = 2 * precisions * recalls / (precisions + recalls)
    fsc = np.mean(fscores)
    fsc_err = 2 * np.std(fscores)

    auc_scores = np.array(aucs)
    auc = np.mean(auc_scores)
    auc_err = 2 * np.std(auc_scores)

    mcorr_scores = np.array(mcorrs)
    mcorr = np.mean(mcorr_scores)
    mcorr_err = 2 * np.std(mcorr_scores)

    print("Mean confusion matrix and error")
    print(mean_conf)
    print(errs_conf)

    print("Recall = {} +/- {}".format(rec, rec_err))
    print("Precision = {} +/- {}".format(prec, prec_err))
    print("F-score = {} +/- {}".format(fsc, fsc_err))
    print("ROC AUC = {} +/- {}".format(auc, auc_err))
    #print("ROC AUC - original = {} +/- {}".format(auc2, auc_err2))
    # print("ROC AUC - concatenated and iterated = {} +/- {}".format(auc3, auc_err3))
    print("MCC = {} +/- {}".format(mcorr, mcorr_err))

In [34]:
def compute_classifier_metrics(labels, probs, labels1, probs1, threshold=0.5):
    """
    Computes metric: precision, recall, mcc, f1
    :param labels: ground truth
    :param probs: predicted values
    :param labels1:
    :param probs1:
    :param threshold: binding/non-binding threshold
    :return:
    """
    matrices = []
    matrices1 = []

    aucs1 = []
    aucs2 = []

    mcorrs = []
    mcorrs1 = []

    #print("labels", labels)
    #print("probs", probs)

    for l, p in zip(labels, probs):
        #print("l", l)
        #print("p", p)
        aucs2.append(roc_auc_score(l, p))
        l_pred = (p > threshold).astype(int)
        matrices.append(confusion_matrix(l, l_pred))
        mcorrs.append(matthews_corrcoef(l, l_pred))

    for l1, p1 in zip(labels1, probs1):
        #print("in for")
        #print("l1", l1)
        #print("p1", p1)
        aucs1.append(roc_auc_score(l1, p1))
        l_pred1 = (p1 > threshold).astype(int)
        matrices1.append(confusion_matrix(l1, l_pred1))
        mcorrs1.append(matthews_corrcoef(l1, l_pred1))

    print("Metrics with the original version")
    helper_compute_metrics(matrices=matrices,  aucs=aucs2, mcorrs =mcorrs)
    print("Metrics with probabilities concatenated")
    helper_compute_metrics(matrices=matrices1, aucs=aucs1, mcorrs=mcorrs1)

In [35]:
compute_classifier_metrics(labels, class_probabilities, labels1, class_probabilities1, threshold=0.5)

Metrics with the original version
Mean confusion matrix and error
[[27219.1  8506.9]
 [ 2114.2 12063.8]]
[[741.07513789 741.07513789]
 [251.34645412 251.34645412]]
Recall = 0.850881647623078 +/- 0.017727920307701504
Precision = 0.5866460793814928 +/- 0.016378794445637904
F-score = 0.694385120592492 +/- 0.006961589870773184
ROC AUC = 0.8771370307227058 +/- 0.0036987674265059993
MCC = 0.5615496431071428 +/- 0.009644567420419007
Metrics with probabilities concatenated
Mean confusion matrix and error
[[27219.1  8506.9]
 [ 2114.2 12063.8]]
[[741.07513789 741.07513789]
 [251.34645412 251.34645412]]
Recall = 0.850881647623078 +/- 0.017727920307701504
Precision = 0.5866460793814928 +/- 0.016378794445637904
F-score = 0.694385120592492 +/- 0.006961589870773184
ROC AUC = 0.8771370307227058 +/- 0.0036987674265059993
MCC = 0.5615496431071428 +/- 0.009644567420419007
